In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path_map = "/content/drive/MyDrive/PYTHON_DIPLOMA/Maps/Maps/AdM-01-2014.png"
path_flight = "/content/drive/MyDrive/PYTHON_DIPLOMA/Flight_1/Flight 1/image000075.png"

In [ ]:
!git clone --quiet https://github.com/cvg/LightGlue/
!pip install --progress-bar off --quiet -e .
!pip install kornia

In [ ]:
from google.colab.patches import cv2_imshow
import cv2
%cd LightGlue
from lightglue import LightGlue, SuperPoint, ALIKED, DISK, SIFT, match_pair
from lightglue.utils import load_image, rbd
from lightglue import viz2d
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms as transforms

/content/LightGlue


In [ ]:
def get_map_pic_2(mx, my, mdx, mdy):
  path_map = "/content/drive/MyDrive/PYTHON_DIPLOMA/Maps/Maps/AdM-01-2014.png"
  map = cv2.imread(path_map, cv2.IMREAD_UNCHANGED)
  map = map[int(mx):int(mx+mdx), int(my):int(my+mdy)]
  return map

In [ ]:
uav_2 = cv2.imread(path_flight, cv2.IMREAD_UNCHANGED)
uav_2 = cv2.rotate(uav_2, cv2.ROTATE_90_COUNTERCLOCKWISE)
uav_2 = cv2.cvtColor(uav_2, cv2.COLOR_BGR2GRAY)

# **SIFT**

In [ ]:
final_coords = []
for i in np.arange(1.3, 1.9, 0.05):
  image1 = get_map_pic_2(1000*i, 1000*i, 4000/i, 4000/i)
  image2 = cv2.imread(path_flight, cv2.IMREAD_GRAYSCALE)
  sift = cv2.SIFT_create()
  keypoints1, descriptors1 = sift.detectAndCompute(get_map_pic_2(1000*i, 1000*i, 4000/i, 4000/i), None)
  keypoints2, descriptors2 = sift.detectAndCompute(image2, None)
  matcher = cv2.BFMatcher()
  matches = matcher.knnMatch(descriptors1, descriptors2, k=2)
  good_matches = []
  for m, n in matches:
      if m.distance < 0.75 * n.distance:
          good_matches.append(m)

  src_points = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
  dst_points = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

  M, mask = cv2.findHomography(src_points, dst_points, cv2.RANSAC)

  scale = np.sqrt(np.linalg.det(M[:2, :2]))
  angle = np.arctan2(M[1, 0], M[0, 0]) * 180 / np.pi
  translation = M[:, 2]

  print("Scale:", scale)
  print("Rotation angle (degrees):", angle)
  print("Translation:", translation)

  height, width = image1.shape[:2]
  center = np.array([width / 2, height / 2, 1])

  transformed_center = np.dot(M, center)
  transformed_center /= transformed_center[2]
  final_coords.append((transformed_center[0]+1000*i, transformed_center[1]+1000*i))
  print("Transformed center:", final_coords)

In [ ]:
true_point = (2648, 2089)
distances = [np.linalg.norm(np.array(point) - np.array(true_point)) for point in final_coords]
print(distances)
sizes = [4000/i for i in np.arange(1.3, 1.9, 0.05)]
print(sizes)
plt.xlabel('Размер фото (pix)')
plt.ylabel('AED (pix)')
plt.plot(sizes, distances)
plt.show()

# **ORB**

In [ ]:
final_coords = []
for i in np.arange(1.3, 1.9, 0.05):
  image1 = get_map_pic_2(1000*i, 1000*i, 4000/i, 4000/i)
  image2 = cv2.imread(path_flight, cv2.IMREAD_GRAYSCALE)
  orb = cv2.ORB_create()
  keypoints1, descriptors1 = orb.detectAndCompute(get_map_pic_2(1000*i, 1000*i, 4000/i, 4000/i), None)
  keypoints2, descriptors2 = orb.detectAndCompute(image2, None)
  matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
  matches = matcher.match(descriptors1, descriptors2)
  good_matches = []
  for m in matches:
    if m.distance < 75:
        good_matches.append(m)

  src_points = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
  dst_points = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
  print(src_points)
  print(dst_points)
  M, mask = cv2.findHomography(src_points, dst_points, cv2.RANSAC)

  scale = np.sqrt(np.linalg.det(M[:2, :2]))
  angle = np.arctan2(M[1, 0], M[0, 0]) * 180 / np.pi
  translation = M[:, 2]

  print("Scale:", scale)
  print("Rotation angle (degrees):", angle)
  print("Translation:", translation)

  height, width = image1.shape[:2]
  center = np.array([width / 2, height / 2, 1])

  transformed_center = np.dot(M, center)
  transformed_center /= transformed_center[2]
  final_coords.append((transformed_center[0]+1000*i, transformed_center[1]+1000*i))
  print("Transformed center:", final_coords)

In [ ]:
true_point = (2648, 2089)
distances = [np.linalg.norm(np.array(point) - np.array(true_point)) for point in final_coords]
print(distances)
sizes = [4000/i for i in np.arange(1.3, 1.9, 0.05)]
plt.xlabel('Размер фото (pix)')
plt.ylabel('AED (pix)')
plt.plot(sizes, distances)
plt.show()

# **FAST+BRIEF**

In [ ]:
final_coords = []
for i in np.arange(1.3, 1.9, 0.05):
  brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()
  image1 = get_map_pic_2(1000*i, 1000*i, 4000/i, 4000/i)
  image2 = cv2.imread(path_flight, cv2.IMREAD_GRAYSCALE)
  fast = cv2.FastFeatureDetector_create()
  keypoints1 = fast.detect(image1, None)
  keypoints2 = fast.detect(image2, None)
  keypoints1, descriptors1 = brief.compute(image1, keypoints1)
  keypoints2, descriptors2 = brief.compute(image2, keypoints2)
  bf = cv2.BFMatcher()
  matches = bf.knnMatch(descriptors1, descriptors2, k=2)
  good_matches = []
  for m, n in matches:
      if m.distance < 0.75 * n.distance:
          good_matches.append(m)

  src_points = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
  dst_points = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
  M, mask = cv2.findHomography(src_points, dst_points, cv2.RANSAC)

  scale = np.sqrt(np.linalg.det(M[:2, :2]))
  angle = np.arctan2(M[1, 0], M[0, 0]) * 180 / np.pi
  translation = M[:, 2]

  print("Scale:", scale)
  print("Rotation angle (degrees):", angle)
  print("Translation:", translation)

  height, width = image1.shape[:2]
  center = np.array([width / 2, height / 2, 1])

  transformed_center = np.dot(M, center)
  transformed_center /= transformed_center[2]
  final_coords.append((transformed_center[0]+1000*i, transformed_center[1]+1000*i))
  print("Transformed center:", final_coords)

In [ ]:
true_point = (2648, 2089)
distances = [np.linalg.norm(np.array(point) - np.array(true_point)) for point in final_coords]
print(distances)
sizes = [4000/i for i in np.arange(1.3, 1.9, 0.05)]
plt.xlabel('Размер фото (pix)')
plt.ylabel('AED (pix)')
plt.plot(sizes, distances)
plt.show()

# **FAST+SIFT**

In [ ]:
final_coords = []
for i in np.arange(1.3, 1.9, 0.05):
  sift = cv2.SIFT_create()
  image1 = get_map_pic_2(1000*i, 1000*i, 4000/i, 4000/i)
  image2 = cv2.imread(path_flight, cv2.IMREAD_GRAYSCALE)
  fast = cv2.FastFeatureDetector_create()
  keypoints1 = fast.detect(image1, None)
  keypoints2 = fast.detect(image2, None)
  keypoints1, descriptors1 = sift.compute(image1, keypoints1)
  keypoints2, descriptors2 = sift.compute(image2, keypoints2)
  bf = cv2.BFMatcher()
  matches = bf.knnMatch(descriptors1, descriptors2, k=2)
  good_matches = []
  for m, n in matches:
      if m.distance < 0.75 * n.distance:
          good_matches.append(m)

  src_points = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
  dst_points = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
  M, mask = cv2.findHomography(src_points, dst_points, cv2.RANSAC)

  scale = np.sqrt(np.linalg.det(M[:2, :2]))
  angle = np.arctan2(M[1, 0], M[0, 0]) * 180 / np.pi
  translation = M[:, 2]

  print("Scale:", scale)
  print("Rotation angle (degrees):", angle)
  print("Translation:", translation)

  height, width = image1.shape[:2]
  center = np.array([width / 2, height / 2, 1])

  transformed_center = np.dot(M, center)
  transformed_center /= transformed_center[2]
  final_coords.append((transformed_center[0]+1000*i, transformed_center[1]+1000*i))
  print("Transformed center:", final_coords)

In [ ]:
true_point = (2648, 2089)
final_coords = [(1748.4893794646239, 1478.8790219908367), (1629.673686973992, 1412.2153311348732), (1679.7770728733587, 1462.35135607735), (1702.7838066791269, 1674.009236370458), (1779.589909684702, 1562.1051052624246), (1829.2889571520634, 1611.7091416813998), (1878.836964245516, 1661.1144541128833), (1928.2572022919344, 1710.3516606674555), (1954.9633797536653, 1923.5277357413545)]
distances = [np.linalg.norm(np.array(point) - np.array(true_point)) for point in final_coords]
print(distances)
sizes = [4000/i for i in np.arange(1.3, 1.9, 0.05)]
plt.xlabel('Размер фото (pix)')
plt.ylabel('AED (pix)')
plt.plot(sizes, distances)
plt.show()

# **STAR+SIFT**

In [ ]:
star = cv2.xfeatures2d.StarDetector_create()
kp_uav = star.detect(uav_2, None)
sift = cv2.SIFT_create()
kp_uav, des_uav = sift.compute(uav_2, kp_uav)
final_coords = []
for i in np.arange(1.3, 1.9, 0.05):
  matched_points_x = []
  matched_points_y = []
  map = get_map_pic_2(1000*i, 1000*i, 4000/i, 4000/i)
  kp_sat = star.detect(map, None)
  kp_sat, des_sat = sift.compute(map, kp_sat)
  bf = cv2.BFMatcher()
  matches = bf.knnMatch(des_sat, des_uav, k=2)

  good = []
  for m, n in matches:
      if m.distance < 0.75 * n.distance:
          good.append([m])


  final_coords.append((mean_x + 1000*i, mean_y + 1000*i))
  matches_img = cv2.drawMatchesKnn(map, kp_sat, uav_2, kp_uav, good, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
  cv2_imshow(cv2.resize(matches_img, (0,0), fx=0.25, fy=0.25))

In [ ]:
true_point = (2648, 2089)
distances = [np.linalg.norm(np.array(point) - np.array(true_point)) for point in final_coords]
print(distances)
sizes = [4000/i for i in np.arange(1.3, 1.9, 0.05)]
plt.xlabel('Размер фото (pix)')
plt.ylabel('AED (pix)')
plt.plot(sizes, distances)
plt.show()

# **STAR+ORB**

In [ ]:
star = cv2.xfeatures2d.StarDetector_create()
kp_uav = star.detect(uav_2, None)
orb = cv2.ORB_create()
kp_uav, des_uav = orb.compute(uav_2, kp_uav)
final_coords = []
for i in np.arange(1.3, 1.9, 0.05):
  matched_points_x = []
  matched_points_y = []
  map = get_map_pic_2(1000*i, 1000*i, 4000/i, 4000/i)
  kp_sat = star.detect(map, None)
  kp_sat, des_sat = orb.compute(map, kp_sat)
  bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
  matches = bf.match(des_sat, des_uav)

  for m in matches:
    if m.distance < 75:
        good_matches.append(m)

  final_coords.append((mean_x + 1000*i, mean_y + 1000*i))
  matches_img = cv2.drawMatchesKnn(map, kp_sat, uav_2, kp_uav, good, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
  cv2_imshow(cv2.resize(matches_img, (0,0), fx=0.25, fy=0.25))

In [ ]:
true_point = (2648, 2089)
distances = [np.linalg.norm(np.array(point) - np.array(true_point)) for point in final_coords]
print(distances)
sizes = [4000/i for i in np.arange(1.3, 1.9, 0.05)]
plt.xlabel('Размер фото (pix)')
plt.ylabel('AED (pix)')
plt.plot(sizes, distances)
plt.show()

# **STAR+BRIEF**

In [ ]:
star = cv2.xfeatures2d.StarDetector_create()
kp_uav = star.detect(uav_2, None)
brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()
kp_uav, des_uav = brief.compute(uav_2, kp_uav)
final_coords = []
for i in np.arange(1.3, 1.9, 0.05):
  matched_points_x = []
  matched_points_y = []
  map = get_map_pic_2(1000*i, 1000*i, 4000/i, 4000/i)
  kp_sat = star.detect(map, None)
  kp_sat, des_sat = brief.compute(map, kp_sat)
  bf = cv2.BFMatcher()
  matches = bf.knnMatch(des_sat, des_uav, k=2)

  for m, n in matches:
      if m.distance < 0.75 * n.distance:
          good_matches.append(m)

  final_coords.append((mean_x + 1000*i, mean_y + 1000*i))
  matches_img = cv2.drawMatchesKnn(map, kp_sat, uav_2, kp_uav, good, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
  cv2_imshow(cv2.resize(matches_img, (0,0), fx=0.25, fy=0.25))

In [ ]:
true_point = (2648, 2089)
distances = [np.linalg.norm(np.array(point) - np.array(true_point)) for point in final_coords]
print(distances)
sizes = [4000/i for i in np.arange(1.3, 1.9, 0.05)]
plt.xlabel('Размер фото (pix)')
plt.ylabel('AED (pix)')
plt.plot(sizes, distances)
plt.show()

# **KAZE**

In [ ]:
final_coords = []
for i in np.arange(1.3, 1.9, 0.05):
  image1 = get_map_pic_2(1000*i, 1000*i, 4000/i, 4000/i)
  image2 = cv2.imread(path_flight, cv2.IMREAD_GRAYSCALE)
  kaze = cv2.KAZE_create()
  keypoints1, descriptors1 = kaze.detectAndCompute(get_map_pic_2(1000*i, 1000*i, 4000/i, 4000/i), None)
  keypoints2, descriptors2 = kaze.detectAndCompute(image2, None)
  matcher = cv2.BFMatcher()
  matches = matcher.knnMatch(descriptors1, descriptors2, k=2)
  good_matches = []
  for m, n in matches:
      if m.distance < 0.75 * n.distance:
          good_matches.append(m)

  src_points = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
  dst_points = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

  M, mask = cv2.findHomography(src_points, dst_points, cv2.RANSAC)

  scale = np.sqrt(np.linalg.det(M[:2, :2]))
  angle = np.arctan2(M[1, 0], M[0, 0]) * 180 / np.pi
  translation = M[:, 2]

  print("Scale:", scale)
  print("Rotation angle (degrees):", angle)
  print("Translation:", translation)

  height, width = image1.shape[:2]
  center = np.array([width / 2, height / 2, 1])

  transformed_center = np.dot(M, center)
  transformed_center /= transformed_center[2]
  final_coords.append((transformed_center[0]+1000*i, transformed_center[1]+1000*i))
  print("Transformed center:", final_coords)

In [ ]:
true_point = (2648, 2089)
distances = [np.linalg.norm(np.array(point) - np.array(true_point)) for point in final_coords]
print(distances)
sizes = [4000/i for i in np.arange(1.3, 1.9, 0.05)]
plt.xlabel('Размер фото (pix)')
plt.ylabel('AED (pix)')
plt.plot(sizes, distances)
plt.show()

# **AKAZE**

In [ ]:
final_coords = []
for i in np.arange(1.3, 1.9, 0.05):
  image1 = get_map_pic_2(1000*i, 1000*i, 4000/i, 4000/i)
  image2 = cv2.imread(path_flight, cv2.IMREAD_GRAYSCALE)
  akaze = cv2.AKAZE_create()
  keypoints1, descriptors1 = akaze.detectAndCompute(get_map_pic_2(1000*i, 1000*i, 4000/i, 4000/i), None)
  keypoints2, descriptors2 = akaze.detectAndCompute(image2, None)
  matcher = cv2.BFMatcher()
  matches = matcher.knnMatch(descriptors1, descriptors2, k=2)
  good_matches = []
  for m, n in matches:
      if m.distance < 0.75 * n.distance:
          good_matches.append(m)

  src_points = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
  dst_points = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

  M, mask = cv2.findHomography(src_points, dst_points, cv2.RANSAC)

  scale = np.sqrt(np.linalg.det(M[:2, :2]))
  angle = np.arctan2(M[1, 0], M[0, 0]) * 180 / np.pi
  translation = M[:, 2]

  print("Scale:", scale)
  print("Rotation angle (degrees):", angle)
  print("Translation:", translation)

  height, width = image1.shape[:2]
  center = np.array([width / 2, height / 2, 1])

  transformed_center = np.dot(M, center)
  transformed_center /= transformed_center[2]
  final_coords.append((transformed_center[0]+1000*i, transformed_center[1]+1000*i))
  print("Transformed center:", final_coords)

In [ ]:
true_point = (2648, 2089)
distances = [np.linalg.norm(np.array(point) - np.array(true_point)) for point in final_coords]
print(distances)
sizes = [4000/i for i in np.arange(1.3, 1.9, 0.05)]
plt.xlabel('Размер фото (pix)')
plt.ylabel('AED (pix)')
plt.plot(sizes, distances)
plt.show()